In [ ]:
get_ipython().system('pip install google-cloud-bigquery')
get_ipython().system('pip install google-cloud-bigquery[pandas]')
get_ipython().system('python3 -m pip install google.cloud')
get_ipython().system('python3 -m pip install pandas')
get_ipython().system('python3 -m pip install numpy')
get_ipython().system('python3 -m pip install scikit-learn')
get_ipython().system('python3 -m pip install plotly')

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt  
import os
import seaborn as sns
from pandas import Series, DataFrame

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt  
import os
import seaborn as sns
from pandas import Series, DataFrame

In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
unemployment_data = query_job.to_dataframe()
unemployment_data.head()

In [ ]:
query2 = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job2 = bigquery_client.query(query2)
wage_data = query_job2.to_dataframe()
wage_data.head()

In [ ]:
wage_data[wage_data.isnull().any(axis=1)]

In [ ]:
wage_data2 = wage_data.fillna({'average_wage': wage_data.average_wage.mean()})
wage_data2.head()

In [ ]:
query3 = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job3 = bigquery_client.query(query3)
prediction_list = query_job3.to_dataframe()
prediction_list.head()

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.INFORMATION_SCHEMA.TABLES`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()
data.head()

In [ ]:
#Merge the data
unemployment_wage_data = pd.merge(unemployment_data, wage_data, on=['uu_id'], how='inner')
unemployment_wage_data = unemployment_wage_data.drop(['countyfips_y', 'tract_y', 'tract_name_y'], axis=1)
unemployment_wage_data = unemployment_wage_data.fillna(0)
unemployment_wage_data.describe()

In [ ]:
unemployment_wage_data.head()

In [ ]:
#Check for duplicated rows
duplicated_rows = sum(unemployment_wage_data.duplicated()) 
unemployment_wage_data = unemployment_wage_data.drop_duplicates()

In [ ]:
unemployment_wage_data[unemployment_wage_data.isnull().any(axis=1)]

In [ ]:
#heat map for correlations
plt.figure(figsize=(25,10))
cor = unemployment_wage_data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds) 
plt.show()

In [ ]:
unemployment_wage_data.columns

In [ ]:
unemployment_wage_data.dtypes

In [ ]:
X = unemployment_wage_data.drop(['total_claims', 'week_number'], axis = 1)
y = unemployment_wage_data.total_claims

In [ ]:
uuid, label = unemployment_wage_data['uu_id'].factorize(sort=True)

In [ ]:
X['uu_id'] = uuid

In [ ]:
X['tract_name_x'] = X['tract_name_x'].factorize()[0]
X['top_category_employer1'] = X['top_category_employer1'].factorize()[0]
X['top_category_employer2'] = X['top_category_employer2'].factorize()[0]
X['top_category_employer3'] = X['top_category_employer3'].factorize()[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1000, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

In [ ]:
prediction_list

In [ ]:
X['uu_id'] = label[X["uu_id"]]
X

In [ ]:
for col in X.columns[2:]:
    li = []
    for i in prediction_list['uu_id']:
        li.append(X.loc[X['uu_id'] == i, col].mean())
    prediction_list[col] = li
prediction_list

In [ ]:
prediction_list['uu_id'] = prediction_list['uu_id'].factorize(sort=True)[0]

In [ ]:
claims_predict = rf.predict(prediction_list)
claims_predict

In [ ]:
submission_df = pd.DataFrame()

In [ ]:
submission_df["uu_id"] = prediction_list["uu_id"]
submission_df["week_number"] = prediction_list["week_number"]
submission_df["total_claims"] = claims_predict

In [ ]:
submission_df

In [ ]:
submission_df["uu_id"] = label[submission_df["uu_id"]]

In [ ]:
submission_df.to_csv('submission2_prediction_output.csv', index=False)